In [23]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score, plot_roc_curve

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [25]:
X = pd.read_csv("../data/number_df.csv").drop('Unnamed: 0', axis = 1)
y = pd.read_csv("../data/number_target.csv").drop('Unnamed: 0', axis = 1)

In [26]:
X.columns

Index(['approx_payout_date', 'body_length', 'channels', 'delivery_method',
       'event_created', 'event_end', 'event_published', 'event_start',
       'fb_published', 'gts', 'has_analytics', 'has_header', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'object_id', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type', 'venue_latitude',
       'venue_longitude'],
      dtype='object')

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify = y)

In [28]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_hat = dt.predict(X_test)
dt.score(X_test, y_test)
#want close to 1
f1_score(y_test, y_hat)

0.7815384615384615

Bring in Ticket DF and merge

In [29]:
ticket = pd.read_csv('../data/ticket_type_df.csv').drop('Unnamed: 0', axis = 1)
ticket.fillna(0, inplace = True)
ticket = ticket.replace(np.inf, 110)

In [30]:
X = pd.concat([X, ticket],axis=1)

In [31]:
X

,approx_payout_date,body_length,channels,delivery_method,event_created,event_end,event_published,event_start,fb_published,gts,...,show_map,user_age,user_created,user_type,venue_latitude,venue_longitude,cost,quantity,num_sold,percent_sold
0,1266062400,3852,5,0.0,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,...,1,36,1259613950,1,25.777471,-80.133433,208.333333,920.0,0.0,0.000000
1,1296720000,3499,0,1.0,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,...,0,149,1280942776,3,32.776566,-79.930922,35.000000,100.0,25.0,0.250000
2,1296172800,2601,8,1.0,1291090956,1295740800,1.291092e+09,1295713800,0,3500.00,...,0,214,1272559388,3,33.944201,-118.080419,93.510000,48.0,48.0,1.000000
3,1388966400,12347,6,1.0,1360681570,1388534400,1.360683e+09,1360702800,0,1167.35,...,0,889,1283870102,3,0.000000,0.000000,13.666667,30000.0,58.0,0.001933
4,1297900800,2417,11,0.0,1291994666,1297468800,1.291995e+09,1297440000,1,2313.15,...,0,35,1288984065,3,42.353848,-71.044276,101.750000,264.0,39.0,0.147727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332,1361689200,667,0,0.0,1360297993,1361257200,1.360299e+09,1361232000,0,0.00,...,1,0,1360297993,1,39.373780,-76.629921,45.000000,400.0,0.0,0.000000
14333,1365566400,5812,5,1.0,1360367042,1365134400,1.360370e+09,1365123600,0,9626.30,...,1,1374,1241625346,4,32.778906,-117.209791,43.250000,3256.0,628.0,0.192875
14334,1368781200,557,13,0.0,1360600330,1368349200,1.361337e+09,1368327600,1,22.09,...,1,84,1353386971,4,30.041819,-89.957130,42.000000,148.0,1.0,0.006757
14335,1361361600,10873,8,0.0,1360376285,1360929600,1.360377e+09,1360890000,0,170.00,...,1,812,1290204991,3,40.862284,-73.911363,79.330000,100.0,0.0,0.000000


Random Forest with Weights

In [137]:
def balance_work(y_train):
    n1 = np.sum(y_train)
    n2 = len(y_train) - n1
    n_samples = n1 + n2
    w1 = n_samples / (2 * n1)
    w2 = n_samples / (2 * n2)
    return w1, w2

In [138]:
def Random_forest_model(X, y, num_trees, num_features):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    w1, w2 = balance_work(y_train)
    rf_model = RandomForestClassifier(n_estimators = num_trees, max_features = num_features, class_weight={1: w1, 0: w2})
    rf_model.fit(X_train, y_train.values.ravel())
    y_predict = rf_model.predict(X_test)
    score = rf_model.score(X_test,y_test)
    return score, confusion_matrix(y_test, y_predict), f1_score(y_test, y_predict), rf_model
rf_score2, rf_matrix2, f1_2, model2 = Random_forest_model(X, y, 50, 'sqrt')

In [139]:
rf_score2

0.9874476987447699

In [140]:
rf_matrix2.ravel()

array([3255,    7,   38,  285])

In [141]:
f1_2

0.926829268292683

Oversample

In [32]:
def div_count_pos_neg(X, y):
    negatives, positives = y == 0, y == 1
    negative_count, positive_count = np.sum(negatives), np.sum(positives)
    X_positives, y_positives = X[positives], y[positives]
    X_negatives, y_negatives = X[negatives], y[negatives]
    return negative_count, positive_count, X_positives, \
           X_negatives, y_positives, y_negatives

In [33]:
def oversample(X, y, tp):
    """Randomly choose positive observations from X & y, with replacement
    to achieve the target proportion of positive to negative observations.

    Parameters
    ----------
    X  : ndarray - 2D
    y  : ndarray - 1D
    tp : float - range [0, 1], target proportion of positive class observations

    Returns
    -------
    X_undersampled : ndarray - 2D
    y_undersampled : ndarray - 1D
    """
#     if (0.5 < np.mean(y)).bool():
#         return X, y
    if 0.5 < np.mean(y):
        return X, y
    neg_count, pos_count, X_pos, X_neg, y_pos, y_neg = div_count_pos_neg(X, y)
    positive_range = np.arange(pos_count)
    positive_size = (tp * neg_count) / (1 - tp)
    positive_idxs = np.random.choice(a=positive_range,
                                     size=int(positive_size),
                                     replace=True)
    X_positive_oversampled = X_pos[positive_idxs]
    y_positive_oversampled = y_pos[positive_idxs]
    X_oversampled = np.vstack((X_positive_oversampled, X_neg))
    y_oversampled = np.concatenate((y_positive_oversampled, y_neg))

    return X_oversampled, y_oversampled

In [34]:
x_o, y_o = oversample(X.values, np.ravel(y.values), 0.5)

In [35]:
x_o.shape, y_o.shape

((26088, 31), (26088,))

In [36]:
def Random_forest_model(X, y, num_trees, num_features):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    rf_model = RandomForestClassifier(n_estimators = num_trees, max_features = num_features)
    rf_model.fit(X_train, y_train)
    y_predict = rf_model.predict(X_test)
    score = rf_model.score(X_test,y_test)
    return score, confusion_matrix(y_test, y_predict), f1_score(y_test, y_predict), rf_model
rf_score3, rf_matrix3, f1_3, model3 = Random_forest_model(x_o, y_o, 50, 'sqrt')

In [37]:
f1_3

0.997858017135863

In [38]:
rf_matrix3.ravel()

array([3247,   14,    0, 3261])

Smote Test

In [39]:
def smote(X, y, tp, k=None):
    """Generates new observations from the positive (minority) class.
    For details, see: https://www.jair.org/media/953/live-953-2037-jair.pdf


    Parameters
    ----------
    X  : ndarray - 2D
    y  : ndarray - 1D
    tp : float - [0, 1], target proportion of positive class observations

    Returns
    -------
    X_smoted : ndarray - 2D
    y_smoted : ndarray - 1D
    """
    if tp < float(np.mean(y)):
        return X, y
    if k is None:
        k = int(len(X) ** 0.5)

    neg_count, pos_count, X_pos, X_neg, y_pos, y_neg = div_count_pos_neg(X, y)
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_pos, y_pos)
    neighbors = knn.kneighbors(return_distance=False)

    positive_size = (tp * neg_count) / (1 - tp)
    smote_num = int(positive_size - pos_count)

    rand_idxs = np.random.randint(0, pos_count, size=smote_num)
    rand_nghb_idxs = np.random.randint(0, k, size=smote_num)
    rand_pcts = np.random.random((smote_num, X.shape[1]))
    smotes = []
    for r_idx, r_nghb_idx, r_pct in zip(rand_idxs, rand_nghb_idxs, rand_pcts):
        rand_pos, rand_pos_neighbors = X_pos[r_idx], neighbors[r_idx]
        rand_pos_neighbor = X_pos[rand_pos_neighbors[r_nghb_idx]]
        rand_dir = rand_pos_neighbor - rand_pos
        rand_change = rand_dir * r_pct
        smoted_point = rand_pos + rand_change
        smotes.append(smoted_point)

    X_smoted = np.vstack((X, np.array(smotes)))
    y_smoted = np.concatenate((y, np.ones((smote_num,))))
    return X_smoted, y_smoted

In [40]:
x_s, y_s = smote(X.values, np.ravel(y.values), 0.5)

In [42]:
def Random_forest_model(X, y, num_trees, num_features):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
    rf_model = RandomForestClassifier(n_estimators = num_trees, max_features = num_features)
    rf_model.fit(X_train, y_train)
    y_predict = rf_model.predict(X_test)
    score = rf_model.score(X_test,y_test)
    return score, confusion_matrix(y_test, y_predict), f1_score(y_test, y_predict), rf_model
rf_score4, rf_matrix4, f1_4, model4 = Random_forest_model(x_s, y_s, 50, 'sqrt')

In [43]:
rf_matrix4.ravel()

array([3234,   27,   28, 3233])

In [44]:
f1_4

0.9915657107805551